# Making Growing Degree Days Rasters

We will use the historical maximum and minimum temperature data in the netCDF form.  The data will be downloaded from the use cal-adapt data repository at 'http://albers.cnr.berkeley.edu/data/noaa/livneh/CA_NV'

In [1]:
# A 'magic' command that displays plots inline inside the notebook as static images.
%matplotlib inline

try:
    #import modules needed for notebook
    import requests 
    #import json
    #import pandas as pd
    import numpy as np
    #import matplotlib.pyplot as plt
    import netCDF4
    import datetime as dt
    from netCDF4 import date2num,num2date
    import os
    #from osgeo import gdal
    #from osgeo import osr

except ImportError:
    print('Some required Python modules are missing.')

### Definition of Growing Degree Days and different methods for calculating.

https://en.wikipedia.org/wiki/Growing_degree-day

GDD are calculated by taking the integral of warmth above a base temperature,[1] Tbase (usually 10 °C):

GDDs are typically measured from the winter low. Any temperature below Tbase is set to Tbase before calculating the average. Likewise, the maximum temperature is usually capped at 30 °C because most plants and insects do not grow any faster above that temperature. However, some warm temperate and tropical plants do have significant requirements for days above 30 °C to mature fruit or seeds.

Example of GDD calculation
For example, a day with a high of 23 °C and a low of 12 °C (and a base of 10 °C) would contribute 7.5 GDDs.

{\displaystyle {\frac {23+12}{2}}-10=7.5} {\frac  {23+12}{2}}-10=7.5
As a second example, a day with a high of 13 °C and a low of 5 °C (and a base of 10 °C) would contribute:

version A: 0 GDD, as:  max((13+5)/2-10,0)=0

version B: 1.5 GDDs, as: (13+10)/2-10=11.5-10=1.5

In [2]:
#  This function downloads data from the given api and saves the file to a
#  local folder defines by the output location (outLoc)
def downloadData(baseurl, year, outLoc, var, fileConcat):
    try:
        filename = fileConcat % (var, year, year)
        url = '%s/%s/%s' % (baseurl, var, filename)
        r = requests.get(url)
        filename = '%s/%s' % (outLoc, filename)
        
        with open(filename, 'wb') as f:
            f.write(r.content)
            
        return filename

    except requests.exceptions.HTTPError as err:
        print(err)

In [3]:
def GDD (x1, x2, baseC, calcType, x3):
    try:
        np.warnings.filterwarnings('ignore')
        if calcType == 'B':
            x1[x1<baseC] = baseC
            x2[x2<baseC] = baseC

        x1[x1>30] = 30
        x2[x2>30] = 30

        x4 = ((x1 + x2)/2 - baseC)
        x4[x4<0] = 0

        x5 = np.add(x3,x4)

        return x5
    except:
        pass

In [4]:
ncfile = netCDF4.Dataset('new1.nc',mode='w',format='NETCDF4_CLASSIC')
lat_dim = ncfile.createDimension('lat', 195) # latitude axis
lon_dim = ncfile.createDimension('lon', 179) # longitude axis
time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).

ncfile.title='My model data'
ncfile.subtitle="My model data subtitle"
ncfile.anything="write anything"

lat = ncfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'

lon = ncfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'

nlats = len(lat_dim); nlons = len(lon_dim); ntimes = 3

time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'days since 1900-01-01 00:00:00'
time.long_name = 'time'

gdd = ncfile.createVariable('gdd',np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost
gdd.units = 'C' # degrees Kelvin
gdd.standard_name = 'Growing Degree Days Base 10 C' # this is a CF standard name

gdd_400 = ncfile.createVariable('gdd_400',np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost
gdd_400.units = 'Julian Date' # degrees Kelvin
gdd_400.standard_name = 'Julain Date of Growing Degree Days Base 10 C >= 400' # this is a CF standard name

In [5]:
# entry point of the API
baseurl = 'http://albers.cnr.berkeley.edu/data/scripps/loca/met/HadGEM2-ES/historical'
fileConcat = '%s_day_HadGEM2-ES_historical_r1i1p1_%s0101-%s1231.LOCA_2016-04-02.16th.CA_NV.nc'
values = [[baseurl, fileConcat, range(1950,2006)]]

baseurl = 'http://albers.cnr.berkeley.edu/data/scripps/loca/met/HadGEM2-ES/rcp85'
fileConcat = '%s_day_HadGEM2-ES_rcp85_r1i1p1_%s0101-%s1231.LOCA_2016-04-02.16th.CA_NV.nc'
values.append([baseurl, fileConcat, range(2006,2100)])

outLoc = 'Data'
count1 = 0
dates = []
netCount = 0
GDDaysThreshold = 400

try:
    for j in values:
        baseurl = j[0]
        fileConcat = j[1]

        for year in j[2]:
            filename = downloadData(baseurl, year, outLoc, "tasmax", fileConcat)
            fh1 = netCDF4.Dataset(filename, mode='r')

            lat[:] = fh1.variables["lat"][:].filled()
            lon[:] = fh1.variables["lon"][:].filled()

            filename2 = downloadData(baseurl, year, outLoc, "tasmin", fileConcat)
            fh2 = netCDF4.Dataset(filename2, mode='r')

            lCount = 0

            tmin = fh2.variables['tasmin']
            tmax = fh1.variables['tasmax']
            lastDate = len(tmax) - 1
            timecnt = fh1.variables["time"][lastDate]

            for i in range(0,len(tmax)):
                x1 = tmax[i] - 273.15
                x2 = tmin[i] - 273.15

                lCount = lCount + 1
                if lCount == 1:
                    GDDays = x2 * 0                    
                    GDDays[GDDays>100000] = np.nan
                    threshold = x2 * 0
                    threshold[threshold>100000] = np.nan
                GDDays = GDD(x1,x2, 10, 'A', GDDays)
                test = np.where(GDDays >= GDDaysThreshold, lCount, 0)
                threshold = np.where(threshold == 0, test, threshold)

            GDDays = GDDays.filled()
            GDDays.shape = (195,179)
            gdd[netCount,:,:] = GDDays # Appends data along unlimited dimension
            
            threshold[threshold == 0] = np.nan
            threshold.shape = (195,179)
            gdd_400[netCount,:,:] = threshold # Appends data along unlimited dimension

            dates = np.append(dates, num2date(timecnt, time.units))

            fh1.close()
            fh1 = ""
            os.remove(filename)

            fh2.close()
            fh2 = ""
            os.remove(filename2)
            netCount = netCount + 1
            
    time[:] = date2num(dates, units = time.units)

    ncfile.close(); print('Dataset is closed!')

except Exception as ex:
    print(ex)

Dataset is closed!


In [6]:
path = 'albers.cnr.berkeley.edu/data/'
# NOAA Data Netcdf
source = 'noaa'
type_ = 'livneh/CA_NV'
root = 'livneh_CA_NV_15Oct2014.%s%s.nc' % (year, month)
fullpath = 'http://albers.cnr.berkeley.edu/data/noaa/livneh/CA_NV/livneh_CA_NV_15Oct2014.195001.nc'

# NOAA Data Netcdf
source = 'noaa'
type_ = 'livneh/CA_NV'
root = 'livneh_CA_NV_15Oct2014.%s%s.nc' % (year, month)
fullpath = 'http://albers.cnr.berkeley.edu/data/noaa/livneh/CA_NV/livneh_CA_NV_15Oct2014.195001.nc'

http://albers.cnr.berkeley.edu/data/scripps/loca/met/ACCESS1-0/historical/tasmax/tasmax_day_ACCESS1-0_historical_r1i1p1_19500101-19501231.LOCA_2016-04-02.16th.CA_NV.nc

SyntaxError: invalid syntax (<ipython-input-6-933ba83747dc>, line 14)